In [0]:
%pip install faker
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Cell 3: Generate Synthetic Fraud Data
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random

fake = Faker()
Faker.seed(42)
np.random.seed(42)

# Configuration
NUM_PATIENTS = 1000
NUM_PROVIDERS = 50
NUM_CLAIMS = 5000
FRAUD_RATE = 0.08  # 8% fraud

# Specialties and associated procedures
SPECIALTIES = {
    'Cardiology': ['Angioplasty', 'ECG', 'Stress Test', 'Cardiac Catheterization', 'Pacemaker Insertion'],
    'Oncology': ['Chemotherapy', 'Radiation Therapy', 'Biopsy', 'CT Scan', 'PET Scan'],
    'Orthopedics': ['Hip Replacement', 'Knee Replacement', 'Fracture Repair', 'Arthroscopy', 'Spinal Fusion'],
    'Neurology': ['MRI Brain', 'EEG', 'Lumbar Puncture', 'EMG', 'Nerve Conduction Study'],
    'General Surgery': ['Appendectomy', 'Hernia Repair', 'Gallbladder Removal', 'Colonoscopy', 'Endoscopy']
}

# Diagnosis codes
DIAGNOSES = {
    'Cardiology': ['I21.9', 'I50.9', 'I48.91', 'I25.10'],
    'Oncology': ['C50.9', 'C61', 'C34.90', 'C18.9'],
    'Orthopedics': ['M17.9', 'S72.9', 'M25.50', 'M54.5'],
    'Neurology': ['G40.9', 'G43.9', 'G20', 'G35'],
    'General Surgery': ['K35.80', 'K40.90', 'K80.20', 'K92.2']
}

# Typical price ranges
PROCEDURE_COSTS = {
    'Angioplasty': (15000, 35000), 'ECG': (200, 500), 'Stress Test': (500, 1500),
    'Chemotherapy': (5000, 15000), 'Radiation Therapy': (8000, 25000), 'Biopsy': (1000, 3000),
    'Hip Replacement': (30000, 50000), 'Knee Replacement': (25000, 45000), 'Fracture Repair': (5000, 15000),
    'MRI Brain': (2000, 5000), 'EEG': (500, 1500), 'Appendectomy': (8000, 15000),
    'Hernia Repair': (6000, 12000), 'Colonoscopy': (1500, 3500), 'CT Scan': (1000, 3000)
}

print("Generating Patients...")
patients = []
for i in range(NUM_PATIENTS):
    patients.append({
        'patient_id': f'P{i+1:05d}',
        'patient_name': fake.name(),
        'dob': fake.date_of_birth(minimum_age=18, maximum_age=90),
        'gender': random.choice(['M', 'F']),
        'state': fake.state_abbr()
    })
patients_df = pd.DataFrame(patients)

print("Generating Providers...")
providers = []
for i in range(NUM_PROVIDERS):
    specialty = random.choice(list(SPECIALTIES.keys()))
    providers.append({
        'provider_id': f'PR{i+1:04d}',
        'provider_name': f'Dr. {fake.last_name()}',
        'specialty': specialty,
        'state': fake.state_abbr(),
        'npi': fake.numerify(text='##########')
    })
providers_df = pd.DataFrame(providers)

print("Generating Claims with Fraud Patterns...")
claims = []
claim_id_counter = 1

# Helper function to generate normal claim
def generate_normal_claim(patient, provider, claim_date):
    specialty = provider['specialty']
    procedure = random.choice(SPECIALTIES[specialty])
    diagnosis = random.choice(DIAGNOSES[specialty])
    
    min_cost, max_cost = PROCEDURE_COSTS.get(procedure, (1000, 5000))
    billed_amount = round(np.random.uniform(min_cost, max_cost), 2)
    
    return {
        'claim_id': f'C{claim_id_counter:06d}',
        'patient_id': patient['patient_id'],
        'provider_id': provider['provider_id'],
        'claim_date': claim_date,
        'procedure_code': procedure,
        'diagnosis_code': diagnosis,
        'billed_amount': billed_amount,
        'is_fraud': 0,
        'fraud_type': None,
        'fraud_reason': None
    }

# Generate normal claims
num_fraud = int(NUM_CLAIMS * FRAUD_RATE)
num_normal = NUM_CLAIMS - num_fraud

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)

for _ in range(num_normal):
    patient = random.choice(patients)
    provider = random.choice(providers)
    claim_date = fake.date_between(start_date=start_date, end_date=end_date)
    
    claim = generate_normal_claim(patient, provider, claim_date)
    claims.append(claim)
    claim_id_counter += 1

# Generate fraud claims
print(f"Injecting {num_fraud} fraudulent claims...")

fraud_patterns = [
    'duplicate_billing',
    'abnormal_amount',
    'procedure_mismatch',
    'upcoding',
    'unbundling'
]

for _ in range(num_fraud):
    patient = random.choice(patients)
    provider = random.choice(providers)
    claim_date = fake.date_between(start_date=start_date, end_date=end_date)
    fraud_type = random.choice(fraud_patterns)
    
    specialty = provider['specialty']
    procedure = random.choice(SPECIALTIES[specialty])
    diagnosis = random.choice(DIAGNOSES[specialty])
    
    if fraud_type == 'duplicate_billing':
        # Create exact duplicate of a recent claim
        if len(claims) > 10:
            original = random.choice(claims[-100:])
            claim = original.copy()
            claim['claim_id'] = f'C{claim_id_counter:06d}'
            claim['claim_date'] = claim_date
            claim['is_fraud'] = 1
            claim['fraud_type'] = 'duplicate_billing'
            claim['fraud_reason'] = 'Duplicate claim for same patient/procedure'
        else:
            continue
    
    elif fraud_type == 'abnormal_amount':
        # 3-5x normal price
        min_cost, max_cost = PROCEDURE_COSTS.get(procedure, (1000, 5000))
        billed_amount = round(np.random.uniform(min_cost * 3, max_cost * 5), 2)
        claim = {
            'claim_id': f'C{claim_id_counter:06d}',
            'patient_id': patient['patient_id'],
            'provider_id': provider['provider_id'],
            'claim_date': claim_date,
            'procedure_code': procedure,
            'diagnosis_code': diagnosis,
            'billed_amount': billed_amount,
            'is_fraud': 1,
            'fraud_type': 'abnormal_amount',
            'fraud_reason': f'Amount {billed_amount:.0f} is 3-5x normal range'
        }
    
    elif fraud_type == 'procedure_mismatch':
        # Wrong specialty doing procedure
        wrong_specialty = random.choice([s for s in SPECIALTIES.keys() if s != specialty])
        procedure = random.choice(SPECIALTIES[wrong_specialty])
        min_cost, max_cost = PROCEDURE_COSTS.get(procedure, (1000, 5000))
        billed_amount = round(np.random.uniform(min_cost, max_cost), 2)
        claim = {
            'claim_id': f'C{claim_id_counter:06d}',
            'patient_id': patient['patient_id'],
            'provider_id': provider['provider_id'],
            'claim_date': claim_date,
            'procedure_code': procedure,
            'diagnosis_code': diagnosis,
            'billed_amount': billed_amount,
            'is_fraud': 1,
            'fraud_type': 'procedure_mismatch',
            'fraud_reason': f'{specialty} provider performing {wrong_specialty} procedure'
        }
    
    elif fraud_type == 'upcoding':
        # Bill more expensive procedure
        expensive_procedures = ['Hip Replacement', 'Knee Replacement', 'Angioplasty', 'Chemotherapy']
        procedure = random.choice(expensive_procedures)
        min_cost, max_cost = PROCEDURE_COSTS.get(procedure, (20000, 50000))
        billed_amount = round(np.random.uniform(max_cost * 0.9, max_cost * 1.2), 2)
        claim = {
            'claim_id': f'C{claim_id_counter:06d}',
            'patient_id': patient['patient_id'],
            'provider_id': provider['provider_id'],
            'claim_date': claim_date,
            'procedure_code': procedure,
            'diagnosis_code': diagnosis,
            'billed_amount': billed_amount,
            'is_fraud': 1,
            'fraud_type': 'upcoding',
            'fraud_reason': f'Upcoded to expensive procedure: {procedure}'
        }
    
    else:  # unbundling
        # Multiple small charges instead of bundled
        min_cost, max_cost = PROCEDURE_COSTS.get(procedure, (1000, 5000))
        billed_amount = round(np.random.uniform(min_cost * 1.5, max_cost * 2), 2)
        claim = {
            'claim_id': f'C{claim_id_counter:06d}',
            'patient_id': patient['patient_id'],
            'provider_id': provider['provider_id'],
            'claim_date': claim_date,
            'procedure_code': procedure,
            'diagnosis_code': diagnosis,
            'billed_amount': billed_amount,
            'is_fraud': 1,
            'fraud_type': 'unbundling',
            'fraud_reason': 'Unbundled procedures billed separately'
        }
    
    claims.append(claim)
    claim_id_counter += 1

claims_df = pd.DataFrame(claims)

# Add month column for time-based queries
claims_df['claim_month'] = pd.to_datetime(claims_df['claim_date']).dt.to_period('M').astype(str)

print("\n=== DATA GENERATION COMPLETE ===")
print(f"Patients: {len(patients_df)}")
print(f"Providers: {len(providers_df)}")
print(f"Total Claims: {len(claims_df)}")
print(f"Fraudulent Claims: {claims_df['is_fraud'].sum()} ({claims_df['is_fraud'].mean()*100:.1f}%)")
print("\nFraud Type Distribution:")
print(claims_df[claims_df['is_fraud']==1]['fraud_type'].value_counts())

# Display sample data
print("\n=== SAMPLE FRAUDULENT CLAIMS ===")
display(claims_df[claims_df['is_fraud']==1].head(10))

Generating Patients...
Generating Providers...
Generating Claims with Fraud Patterns...
Injecting 400 fraudulent claims...

=== DATA GENERATION COMPLETE ===
Patients: 1000
Providers: 50
Total Claims: 5000
Fraudulent Claims: 400 (8.0%)

Fraud Type Distribution:
fraud_type
duplicate_billing     104
abnormal_amount        83
procedure_mismatch     75
upcoding               73
unbundling             65
Name: count, dtype: int64

=== SAMPLE FRAUDULENT CLAIMS ===


claim_id,patient_id,provider_id,claim_date,procedure_code,diagnosis_code,billed_amount,is_fraud,fraud_type,fraud_reason,claim_month
C004601,P00249,PR0050,2024-11-15,Stress Test,I25.10,1298.66,1,duplicate_billing,Duplicate claim for same patient/procedure,2024-11
C004602,P00014,PR0002,2024-03-04,Radiation Therapy,C61,11515.9,1,duplicate_billing,Duplicate claim for same patient/procedure,2024-03
C004603,P00675,PR0006,2024-11-28,MRI Brain,G35,9774.0,1,abnormal_amount,Amount 9774 is 3-5x normal range,2024-11
C004604,P00732,PR0021,2024-05-07,Spinal Fusion,M54.5,4765.9,1,duplicate_billing,Duplicate claim for same patient/procedure,2024-05
C004605,P00865,PR0036,2024-04-30,Angioplasty,G40.9,39725.57,1,upcoding,Upcoded to expensive procedure: Angioplasty,2024-04
C004606,P00067,PR0040,2024-08-03,CT Scan,C50.9,2358.13,1,duplicate_billing,Duplicate claim for same patient/procedure,2024-08
C004607,P00026,PR0012,2024-12-11,Knee Replacement,C61,43185.05,1,procedure_mismatch,Oncology provider performing Orthopedics procedure,2024-12
C004608,P00945,PR0026,2024-04-10,Knee Replacement,G20,45498.13,1,upcoding,Upcoded to expensive procedure: Knee Replacement,2024-04
C004609,P00508,PR0039,2024-03-10,Lumbar Puncture,G35,19528.14,1,abnormal_amount,Amount 19528 is 3-5x normal range,2024-03
C004610,P00388,PR0031,2024-07-30,Fracture Repair,M25.50,6718.29,1,duplicate_billing,Duplicate claim for same patient/procedure,2024-07


In [0]:
# Cell 3.5: Create Database Schema
spark.sql("CREATE SCHEMA IF NOT EXISTS fraud_detection")
spark.sql("USE fraud_detection")

print("✅ Schema 'fraud_detection' created and set as default")

✅ Schema 'fraud_detection' created and set as default


In [0]:
# Cell 4: Save Data to Delta Lake Tables
from pyspark.sql import SparkSession

# Convert pandas DataFrames to Spark DataFrames
patients_spark = spark.createDataFrame(patients_df)
providers_spark = spark.createDataFrame(providers_df)
claims_spark = spark.createDataFrame(claims_df)

# Save as Delta Lake tables (Bronze layer - raw data)
print("Saving to Delta Lake...")

patients_spark.write.format("delta").mode("overwrite").saveAsTable("fraud_detection.patients")
providers_spark.write.format("delta").mode("overwrite").saveAsTable("fraud_detection.providers")
claims_spark.write.format("delta").mode("overwrite").saveAsTable("fraud_detection.claims_bronze")

print("\n✅ Delta Lake Tables Created:")
print("  - fraud_detection.patients")
print("  - fraud_detection.providers")
print("  - fraud_detection.claims_bronze")

# Verify the tables
print("\n=== TABLE ROW COUNTS ===")
print(f"Patients: {spark.table('fraud_detection.patients').count()}")
print(f"Providers: {spark.table('fraud_detection.providers').count()}")
print(f"Claims: {spark.table('fraud_detection.claims_bronze').count()}")

# Show sample
print("\n=== SAMPLE FROM CLAIMS TABLE ===")
spark.table('fraud_detection.claims_bronze').show(5)

Saving to Delta Lake...

✅ Delta Lake Tables Created:
  - fraud_detection.patients
  - fraud_detection.providers
  - fraud_detection.claims_bronze

=== TABLE ROW COUNTS ===
Patients: 1000
Providers: 50
Claims: 5000

=== SAMPLE FROM CLAIMS TABLE ===
+--------+----------+-----------+----------+--------------------+--------------+-------------+--------+----------+------------+-----------+
|claim_id|patient_id|provider_id|claim_date|      procedure_code|diagnosis_code|billed_amount|is_fraud|fraud_type|fraud_reason|claim_month|
+--------+----------+-----------+----------+--------------------+--------------+-------------+--------+----------+------------+-----------+
| C003001|    P00114|     PR0006|2024-09-12|Nerve Conduction ...|         G43.9|      3690.81|       0|      NULL|        NULL|    2024-09|
| C003002|    P00363|     PR0003|2024-02-06|   Radiation Therapy|        C34.90|     21543.58|       0|      NULL|        NULL|    2024-02|
| C003003|    P00836|     PR0015|2024-05-20|     Lu